# 其他工具：Grab、PyQuery


* 利用 Grab 套件的存取 HTML 資源
* 利用 PyQuery 套件的解析 HTML 格式


## 作業目標

將之前用 requests + beatifulsoup 實作的方式，改寫成 grab + pyquery，並且比較有哪些地方不同。





## requests + BeautifulSoup


In [1]:
import requests
import os
from bs4 import BeautifulSoup
from PIL import Image

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
# 決定要儲存的資料夾
output_dir = 'downloads'

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定位所有圖片的 tag
image_tags = soup.find(id='main-content').findChildren('a', recursive=False)
print(image_tags)
for img_tag in image_tags:
    if 'imgur' not in img_tag['href']:
        continue
    # 取得所有圖片在第三方服務的 id
    img_url = img_tag['href']+'.jpg'
    img_id=img_tag['href'].split('/')[-1]
    # 組合圖片而非網站的網址
    with requests.get(img_url, stream=True) as r:
        r.raise_for_status()
        # 檢查圖片副檔名
        img = Image.open(r.raw)
        img_savename = '{outdir}/{img_id}.{img_ext}'.format(
            outdir=output_dir, img_id=img_id, img_ext=img.format.lower())
        img.save(img_savename)
        print('Save image {}'.format(img_savename))
    # 對圖片送出請求

[<a href="https://imgur.com/Cgb5oo1" rel="nofollow" target="_blank">https://imgur.com/Cgb5oo1</a>, <a href="https://imgur.com/MgjHY4f" rel="nofollow" target="_blank">https://imgur.com/MgjHY4f</a>, <a href="https://imgur.com/yeoRtAs" rel="nofollow" target="_blank">https://imgur.com/yeoRtAs</a>, <a href="https://imgur.com/wsauNEW" rel="nofollow" target="_blank">https://imgur.com/wsauNEW</a>, <a href="https://imgur.com/IjiETcs" rel="nofollow" target="_blank">https://imgur.com/IjiETcs</a>]
Save image downloads/Cgb5oo1.jpeg
Save image downloads/MgjHY4f.jpeg
Save image downloads/yeoRtAs.jpeg
Save image downloads/wsauNEW.jpeg
Save image downloads/IjiETcs.jpeg


## Grab + PyQuery

In [52]:
from grab import Grab
from pyquery import PyQuery as pq

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
g = Grab()
resp = g.go(url, cookies={'over18': '1'})
doc = pq(resp.body)
output_dir = 'downloads2'
image_tags=doc('a')

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
#print(image_tags.text())
#print(type(image_tags.text()))這樣會把所有資料變成一個string
for link in image_tags.items():#這樣才能取出所有href
    print(image_tags.items())
    print(link.attr['href'])
    if 'https://imgur' not in link.attr['href']:
        continue
    # 取得所有圖片在第三方服務的 id
    img_url = link.attr['href']+'.jpg'
    img_id=link.attr['href'].split('/')[-1]
    # 組合圖片而非網站的網址
    g=Grab()
    r=g.go(img_url)
    # with g.go(img_url) as r:
    #r.raise_for_status()
        # 檢查圖片副檔名
    img = Image.open(r)
    img_savename = '{outdir}/{img_id}.{img_ext}'.format(
            outdir=output_dir, img_id=img_id, img_ext=img.format.lower())
    img.save(img_savename)
    print('Save image {}'.format(img_savename))
    # 對圖片送出請求

<generator object PyQuery.items at 0x00000201FB19B8C8>
/bbs/
<generator object PyQuery.items at 0x00000201FB19B8C8>
/bbs/Beauty/index.html
<generator object PyQuery.items at 0x00000201FB19B8C8>
/about.html
<generator object PyQuery.items at 0x00000201FB19B8C8>
/contact.html
<generator object PyQuery.items at 0x00000201FB19B848>
/bbs/Beauty/index.html
<generator object PyQuery.items at 0x00000201FB19B848>
https://imgur.com/Cgb5oo1


AttributeError: 'Document' object has no attribute 'read'